In [ ]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import pylab

print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)

In [ ]:
defaultSmoothing = 10e-10
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']
for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [ ]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=1000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
_df0.head()
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha() or pattern.match(letter):
            exec('if \'{0}\' not in {1}Alphabets.keys():\n\
                     {2}Alphabets[letter] = {3}Size\n\
                     {4}Size += 1'.format(letter, language, language, language, language))
            sentence = sentence + letter
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print(trainDict.keys())

# UNIGRAMS

In [ ]:
for language in languages:
    exec("%sModel=np.zeros(shape=(%sSize+1))" %(language, language))
    exec('self.{L}Model = np.add(self.{L}Model, defaultSmoothing)'.format(L=language))

start_time = datetime.now()
delta = 0
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-1):
            first = tweet[i]
            exec('index = %sAlphabets[first]'%(language)) #index = esAlphabets[first] #get index of the character from the language dictionary 
            exec('%sModel[index] += 1'%(language)) #increment that index in the language model
for language in languages:
#     exec('%sModelTemp = %sModel[:]'%(language, language))
    exec('%sModelTemp = np.add(%sModelTemp, delta)'%(language, language)) #this is where smoothing happens
    exec('%sModelTemp = np.divide(%sModelTemp, %sModel.sum(axis=0))'%(language, language, language)) #divide all the values by the sum of the row
    exec('%sModelTemp = np.log10(%sModelTemp)'%(language, language))
#     exec('%sModel[:] = %sModelTemp'%(language, language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

# BIGRAMS

In [ ]:
for language in languages:
    exec("%sModel=np.zeros(shape=((%sSize+1),(%sSize+1))) " %(language, language, language))
    exec('{L}Model = np.add({L}Model, defaultSmoothing)'.format(L=language))
    
start_time = datetime.now()
delta = 0
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('%sModel[firstIndex][secondIndex] += 1'%(language))
# here the Model.sum(axis=1) instead of axis=0 for uni/trigrams. Helps to avoid haveing a loop for every row of 2D array.
for language in languages:
    exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
    exec('%sModel = %sModel/%sModel.sum(axis=1)'%(language, language, language)) #divide all the values by the sum of the row
    exec('%sModel = np.log10(%sModel)'%(language, language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

## SAVE MODEL FOR 1D OR 2D

```python
for language in languages:
    exec('np.savetxt(\'{L}ModelBigram.model\', {L}Model, delimiter=\',\', fmt=\'%1.2e\')'.format(L=language))
``` 

In [ ]:
for language in languages:
    exec('np.savetxt(\'{L}ModelBigram.model\', {L}Model, delimiter=\',\', fmt=\'%1.2e\')'.format(L=language))

# TRIGRAMS

In [ ]:
for language in languages:
    exec('{lang}Model=np.zeros(shape=(({lang}Size+1),({lang}Size+1),({lang}Size+1)))'.format(lang=language))#, language, language, language))
    exec('{L}Model = np.add({L}Model, defaultSmoothing)'.format(L=language))
    
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            third = tweet[i+2]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('thirdIndex = %sAlphabets[third]'%(language))
            exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1'%(language))
for language in languages:
    exec('for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[1]):\n\
        {lang}ModelTemp = {lang}Model[x,y,:]\n\
        {lang}ModelTemp = np.add({lang}ModelTemp, delta)\n\
        {lang}ModelTemp = np.divide({lang}ModelTemp, {lang}ModelTemp.sum(axis=0))\n\
        {lang}ModelTemp = np.log10({lang}ModelTemp)\n\
        {lang}Model[x,y,:] = {lang}ModelTemp\n'.format(lang=language))

print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

## SAVE MODEL FOR 3D

```python
for language in languages:
    exec('outfile = open(\'{L}ModelTrigram.model\', \'w\')\n\
    print(\'# Shape \', {L}Model.shape, file=outfile)\n\
    outfile.flush()\n\
    print(\'# To load model - new_data = np.loadtxt(filename)\', file=outfile)\n\
    outfile.flush()\n\
    print(\'# Reshape the data - new_data = new_data.reshape((shape))\', file=outfile)\n\
    outfile.flush()\n\
    for data_slice in {L}Model:\n\
        np.savetxt(outfile, data_slice, delimiter=\',\', fmt=\'%1.2e\')'.format(L=language))
``` 